# Clase 2 - Problemas de costo fijo
## Investigación Operativa 2C 2025 - Departamento de Matemática - FCEN-UBA

### Inicialización

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

In [ ]:
!conda install -q pyscipopt

In [3]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model
# Importamos numpy
import numpy as np

### Fábrica de muebles

Resolver el modelo para optimizar la producción en la fábrica de muebles,
eligiendo un valor apropiado para la constante $M$:

$$
\begin{array}{crl}
\max & 30x_1-165w_1 + 18x_2-100w_2+68x_3-300w_3 &  \\
\text{s.a.:} & 5x_1 + 3x_2 + 15x_3 \leq& 100 \\
  & 2x_1+2x_2+8x_3 \leq & 60 \\
  & 5x_1+x_2+20x_3 \leq & 100 \\
  & x_1+4x_2+12x_3 \leq & 75 \\
  & x_1 \leq & 0.5\displaystyle\sum_{i=1}^3 x_i \\
  & x_i \leq & Mw_i \quad \forall i=1,2,3 \\
  & x_i \geq & 0 \quad \forall i=1,2,3 \\
  & x_i\in &\mathbb{Z} \quad \forall i=1,2,3 \\
  & w_i\in&\{0,1\} \quad \forall i=1,2,3\\
\end{array}
$$


In [4]:
# Vectores con disponibilidad de materiales (la primera coordenada corresponde a mesa, la segunda a silla y la tercera al ropero)
horas_req = np.array([5, 3, 15])
cedro_req = np.array([2, 2, 8])
pino_req = np.array([5, 1, 20])
nogal_req = np.array([1, 4, 12])

# Cantidad disponible de cada material
horas_disp = 100
cedro_disp = 60
pino_disp = 100
nogal_disp = 75

# Ganancia por mueble (la primera coordenada le corresponde a la mesa, etc.)
g = [30, 18, 68]

# Costo de compra de herramientas
c = [165, 100, 300]

In [7]:
model = Model('muebles')

furniture_range = range(len(g))

# Inicializamos un vector vacío para las variables y lo llenamos con las variables
x = np.empty(len(g), dtype='object')
for i in furniture_range:
  x[i] = model.addVar(f'x_{i}', vtype='I')

w = np.empty(len(g), dtype='object')
for i in furniture_range:
  w[i] = model.addVar(f'w_{i}', vtype='B')

# Restricciones de disponibilidad de materiales
model.addCons(sum(x[i]*horas_req[i] for i in furniture_range) <= horas_disp)
model.addCons(sum(x[i]*cedro_req[i] for i in furniture_range) <= cedro_disp)
model.addCons(sum(x[i]*pino_req[i] for i in furniture_range) <= pino_disp)
model.addCons(sum(x[i]*nogal_req[i] for i in furniture_range) <= nogal_disp)

# Restriccion para las sillas
model.addCons(x[1] <= 0.5*sum(x[i] for i in furniture_range))

# Restricciones de costo fijo
# maxima cantidad de muebles que se pueden hacer
M = 100/3 + 1 # la cantidad maxima de muebles son 33 sillas porque solo tenemos 100 horas y cada silla cuesta 3
for i in furniture_range:
  model.addCons(x[i] <= M*w[i])

# Funcion objetivo
model.setObjective(sum(g[i]*x[i] - c[i]*w[i] for i in furniture_range), sense='maximize')

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/clasesPracticas/clase2-my.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 6 chg bounds, 0 chg sides, 3 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 6 chg bounds, 0 chg sides, 3 chg coeffs, 3 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 6 tightened bounds, 0 added holes, 0 changed sides, 3 changed coefficients
 3 implications, 0 cliques
presolved problem has 6 variables (3 bin, 3 int, 0 impl, 0 cont) and 8 constraints
      3 constraints of type <varbound>
      5 constraints of type <linear>
transformed objective value is always integral 